# Mi primer EDA

## 1. Bloque de código para las bibliotecas que vamos a ir necesitando

In [ ]:
import pandas as pd
import os
import json
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import Point

import folium
from branca.element import Element 

from itertools import combinations

# modelado
from sklearn.model_selection import train_test_split #división de datos del modelo ML
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# escalado
from sklearn.preprocessing import StandardScaler # implementar escalado
from sklearn.preprocessing import MinMaxScaler #implementar el escalado
from pickle import dump #dump: Función para guardar objetos en un archivo, en este caso el escalador.

# encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

#entrenamiento
from sklearn.linear_model import LinearRegression

## 2. Almacenamos la base de datos con la que vamos a trabajar y comprobamos que es correcta.

In [ ]:
# Paso 1: Crear carpeta para almacenar datos en crudo
os.makedirs('./data/raw', exist_ok=True)

# Paso 2: Descargar y almacenar el archivo
url = "https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv"
file_path = './data/raw/AB_NYC_2019.csv'
df = pd.read_csv(url)

# Guardar una copia local en la carpeta indicada
df.to_csv(file_path, index=False)

# Paso 3: Cargar el conjunto de datos
# Verificamos las primeras filas para inspeccionar la estructura y composición del conjunto de datos
df = pd.read_csv(file_path)
df.head()


## 3. Conociendo el Data Set 

In [ ]:
# Obtener las dimensiones
print("Filas , Columnas")
df.shape

Obtenemos información sobre los tipos de datos y valores no nulos para más tarde  
 poder clasificarlos, modificarlos o anularnos en caso de no ser necesarios.

In [ ]:
# Obtener información sobre tipos de datos y valores no nulos
df.info()

## 4. Trabajando los duplicados.

Usamos el método duplicated() para detectar duplicados en un DataFrame y
sum() para contar el número de duplicados.

In [ ]:
duplicados = df.duplicated()
num_duplicados = duplicados.sum()
print(f"En este caso en contramos {num_duplicados} duplicados.")


En caso de que haya duplicados los seleccionamos, eliminamos o modificamos a un valor que concuerde con la info del Data

In [ ]:
#Para seleccionar duplicados:

#df_duplicados = df[duplicados]

#método drop_duplicates() para eliminar filas duplicadas. Se puede indicar el conjunto de atributos a considerar.

#df_sin_duplicados = df.drop_duplicates()

# ejemplo control de duplicados:

#df.duplicated().sum()
# sin considerar el id
#df.drop("Id", axis = 1).duplicated().sum()

## 5. Los Nulos

In [ ]:
df.isnull().sum()

Como podemos ver en last_reviews & en reviews_per_month hay bastantes nulos, es posible que prescindamos de estas columnas.

In [ ]:
for column in df.columns:
    if df[column].isnull().sum() != 0:
        print("=======================================================")
        print(f"{column} ==> Missing Values : {df[column].isnull().sum()}, dtypes : {df[column].dtypes}")

Rellenaremos los nulos de reviews_per_month con ceros para poder trabajar esta columna en caso de que sea necesaria y comprobamos.

In [ ]:
df_new = df.copy()
df_new['reviews_per_month'] = df_new['reviews_per_month'].fillna(0)

df_new.isnull().sum()


## 6 Eliminando atributos no relevantes

In [ ]:
df_new = df_new.drop(['id', 'name', 'host_id', 'host_name', 'last_review'], axis=1)
df_new.head()


Vamos a juntar en una sola columna la latitud y la longitud para tener las coordenadas en caso de que pueda ser util

In [ ]:
# Combinar latitude y longitude en una columna 'coordinates' como tuple
df_new['coordinates'] = df_new.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# Eliminar las columnas originales si ya no son necesarias
df_new.drop(['latitude', 'longitude'], axis=1, inplace=True)

# Mostrar el resultado
df_new.head()

Volvemos a chequear los nulos de nuevo, ya tendríamos limpios los datos en este ámbito:

In [ ]:
df_new.isnull().sum().sort_values(ascending = False)


## 6. Variables Categóricas y numéricas - Limpiando el Data

Comprobamos de nuevo las clases de variables para poder luego seleccionarlas, dividirlas y eliminarlas en caso de no ser necesarias

In [ ]:
df_new.dtypes

In [ ]:
# Identificar las variables numéricas
variables_numericas = df_new._get_numeric_data().columns
print("Las variables numéricas son:")
for var in variables_numericas:
    print(f" - {var}")

# Identificar las variables categóricas
variables_categoricas = set(df_new.columns) - set(variables_numericas)
print("\nLas variables categóricas son:")
for var in variables_categoricas:
    print(f" - {var}")


## 7 Análisis de las variables categóricas
-Empezaremos con un conteo

In [ ]:
df_new.neighbourhood.value_counts()

In [ ]:
df_new.room_type.value_counts()

In [ ]:
df_new.neighbourhood_group.value_counts()

In [ ]:
df_new.coordinates.value_counts()

Analizando si necesitamos las coordenadas para trabajar:

In [ ]:


# Calcular el precio promedio total
average_price_total = df_new['price'].mean()

# Calcular el precio promedio por coordenada
price_by_coordinates = df_new.groupby('coordinates')['price'].mean()

# Crear un mapa centrado en Nueva York
ny_map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# Función para asignar colores según el precio promedio
def color_for_price(price):
    if price < 100:
        return 'green'
    elif price < 300:
        return 'orange'
    else:
        return 'red'

# Agregar marcadores para las 500 coordenadas más frecuentes (según precio promedio)
for coord, avg_price in price_by_coordinates.head(500).items():
    folium.CircleMarker(
        location=[coord[0], coord[1]],  # Coordenadas (lat, long)
        radius=avg_price / 50,  # Ajustar el divisor para reducir el tamaño del marcador
        color=color_for_price(avg_price),  # Color del borde según el precio
        fill=True,
        fill_color=color_for_price(avg_price),  # Color de relleno
        fill_opacity=0.6,
        tooltip=f"<b>Coordenadas:</b> {coord}<br><b>Precio Promedio:</b> ${avg_price:.2f}"
    ).add_to(ny_map)

# Crear una leyenda manual en HTML con el precio promedio global
legend_html = f"""
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 250px;
    height: 150px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
    border-radius: 10px;">
    <b>Rango de Precios</b><br>
    <i style="background:green; width: 10px; height: 10px; float:left; margin-right: 10px; border-radius: 50%;"></i> Menor a $100<br>
    <i style="background:orange; width: 10px; height: 10px; float:left; margin-right: 10px; border-radius: 50%;"></i> $100 a $300<br>
    <i style="background:red; width: 10px; height: 10px; float:left; margin-right: 10px; border-radius: 50%;"></i> Mayor a $300<br>
    <br>
    <b>Precio Promedio Global:</b> ${average_price_total:.2f}
</div>
"""

# Agregar la leyenda al mapa
ny_map.get_root().html.add_child(Element(legend_html))

# Mostrar el mapa interactivo
ny_map


Visualización de la Distribución de Precios:

-Los círculos verdes indican áreas con precios promedio bajos (menores a $100).  
-Los círculos naranjas representan zonas con precios promedio moderados ($100 a $300).  
-Los círculos rojos muestran ubicaciones con precios promedio altos (mayores a $300).  
  
Los círculos rojos más grandes están concentrados en áreas urbanas densas, como Manhattan, lo que refleja una mayor demanda y precios más altos. Aún así podremos encontrar ofertas con precios precios más moderados en casi todas las zonas.  

Teniendo en cuenta que ya contamos con neighbourhood y neighbourhood_group, la extensa lista de coordenadas y que no nos vamos a meter en el análisis de los puntos de interés turístico y la distancia de los airbnb con ellos vamos a prescindir de esta columna para seguir con el EDA. 


In [ ]:
df_new = df_new.drop(['coordinates'], axis=1)
df_new.head()

In [ ]:

categoricas_analisis_1 = ['neighbourhood_group', 'room_type']

# Crear una cuadrícula de gráficos
fig, axes = plt.subplots(nrows=1, ncols=len(categoricas_analisis_1), figsize=(20, 6))

# Iterar por cada variable categórica y graficar con porcentajes
for i, col in enumerate(categoricas_analisis_1):
    # Calcular el porcentaje
    value_counts = df_new[col].value_counts(normalize=True) * 100
    
    # Crear un gráfico de barras con porcentajes
    axes[i].bar(value_counts.index, value_counts.values, color=sns.color_palette("viridis", len(value_counts)))
    
    # Configuración del gráfico
    axes[i].set_title(f'Distribución de {col} (%)')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Porcentaje (%)')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

En los gráficos de arriba podemos ver la cantidad de alojamientos que encontramos en cada agrupación de barrios y la cantidad de tipos de apartamentos / habitación en alquiler. 
Sobresaliendo Manhattan y Brooklyn en localización con más airbnb y los apartamentos completos seguidos de las habitaciones privadas como claras predominantes.

In [ ]:
# Identificar los 20 barrios más representativos
top_20_neighbourhoods = df_new['neighbourhood'].value_counts().head(20).index

# Agrupar los barrios en una serie temporalmente para el gráfico
neighbourhood_counts = df_new['neighbourhood'].apply(
    lambda x: x if x in top_20_neighbourhoods else 'Otros'
).value_counts()

# Graficar la distribución de los barrios agrupados
plt.figure(figsize=(14, 8))
plt.bar(neighbourhood_counts.index, neighbourhood_counts.values, color=sns.color_palette('viridis', len(neighbourhood_counts)))
plt.title('Distribución de Neighbourhood (Top 20 + Otros)')
plt.xlabel('Neighbourhood')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Teniendo en cuenta la cantidad de barrios que hay y que ya los tenemos agrupados en la columna 'neighbourhood_group' he decidido prescindir de esta columna para facilitar el modelo de machine learning aunque renuncie a una preción mayor.

In [ ]:
df_new = df_new.drop(['neighbourhood'], axis=1)
df_new.head()



In [ ]:

# Crear una tabla cruzada para frecuencias absolutas
heatmap_data = pd.crosstab(df_new['neighbourhood_group'], df_new['room_type'])

# Calcular porcentajes
heatmap_percent = heatmap_data.div(heatmap_data.sum(axis=1), axis=0) * 100

# Crear un DataFrame con los porcentajes formateados para anotaciones
annot_percent = heatmap_percent.round(1).astype(str) + '%'  # Formatear como texto con "%"

# Crear una cuadrícula para mostrar ambos gráficos
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Gráfico 1: Heatmap de Frecuencia Absoluta
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="viridis", cbar=True, ax=axes[0])
axes[0].set_title('Frecuencia Absoluta de Room Type por Neighbourhood Group')
axes[0].set_xlabel('Room Type')
axes[0].set_ylabel('Neighbourhood Group')

# Gráfico 2: Heatmap de Porcentajes
sns.heatmap(
    heatmap_percent, 
    annot=annot_percent,  # Usar el DataFrame de porcentajes formateados
    fmt='',  # Para texto personalizado en las anotaciones
    cmap="viridis", 
    cbar=True, 
    ax=axes[1]
)
axes[1].set_title('Porcentaje de Room Type por Neighbourhood Group')
axes[1].set_xlabel('Room Type')
axes[1].set_ylabel('Neighbourhood Group')

plt.tight_layout()
plt.show()


En los gráficos superiores que se representan tanto en porcentaje como en frecuencia podemos observar varias cosas relevantes para nuestro estudio: 
   
- En Manhattan predominan los apartamentos/casas completos con un 60,9% por ciento del mercado. Además de ser la que más propiedades tiene. Podemos sugerir que estamos hablando un enfoque mayor al mercado de lujo.
- En Brooklyn tenemos 50,4% de habitaciones privadas frente el 47,5% de establecimientos completos.   
- En Queens, con un número ya sigificativamente menor de propiedades, podemos ver como también predominan con el 59,5% la habitación privada. Aún siendo una diferencia no muy grande aún.  
- Por último, en el Bronx, con bastantes menos airbnbs sigue predominando la habitación privada. Será en el barrio donde más porcentaje de habitación compartida encontraremos, aún siendo pequeño (5,5%). Nos situamos en zonas con unos presupuestos más ajustados y menos turísticas.

In [ ]:
# Crear una tabla cruzada entre neighbourhood_group y room_type
cross_tab = pd.crosstab(df_new['neighbourhood_group'], df_new['room_type'], normalize='index') * 100

# Graficar la tabla cruzada como barras apiladas
cross_tab.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis')
plt.title('Distribución de Room Type por Neighbourhood Group (%)')
plt.xlabel('Neighbourhood Group')
plt.ylabel('Porcentaje (%)')
plt.legend(title='Room Type')
plt.tight_layout()
plt.show()


## 8 Análisis de las variables numéricas
##### Hagamos un seguimiento de con qué variables nos hemos quedado por ahora

In [ ]:
# Identificar las variables numéricas
variables_numericas = df_new._get_numeric_data().columns
print("Las variables numéricas son:")
for var in variables_numericas:
    print(f" - {var}")

# Identificar las variables categóricas
variables_categoricas = set(df_new.columns) - set(variables_numericas)
print("\nLas variables categóricas son:")
for var in variables_categoricas:
    print(f" - {var}")


Resumen estadístico y gráficos de frecuencia generales de las variables numéricas:

In [ ]:
# Resumen estadístico general
#df_new.describe()


# Configuración de la cuadrícula: histogramas arriba y boxplots abajo
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16, 10), gridspec_kw={'height_ratios': [6, 1]})

# Iterar sobre las variables numéricas
for i, col in enumerate(variables_numericas):
    row = 0  # Fila para el histograma
    col_index = i % 3  # Columna en la cuadrícula
    sns.histplot(data=df_new, x=col, kde=True, bins=30, ax=axes[row, col_index])  # Histograma
    axes[row, col_index].set_title(f'Distribución de {col}')

    row = 1  # Fila para el boxplot
    sns.boxplot(data=df_new, x=col, ax=axes[row, col_index])  # Boxplot
    axes[row, col_index].set_xlabel(col)

# Ajustar diseño para evitar solapamientos
plt.tight_layout()

# Mostrar los gráficos
plt.show()





- Precio: La mayoría de los precios están por debajo de aprox 200$, pero hay valores extremos muy altos.
- Mínimo de noches: La mayor parte de las estancias son cortas pero con los valores extremos no podemos visualizarlo bien.
- Número de reviews: La mayor parte tienen muy pocas reviews y unas pocas destacan. 
- Reseñas por mes: Muchas propiedades no tienen actividad, chequear las que tienen más para identificar patrones. Decido quedarme con el nº total de reviews y elimino esta columna.
- Propiedades por anfitrión: La mayor parte tiene una sola propiedad, omitiremos esta columna. No creo que sea significativa.
- Disponibilidad anual: Relacionar con precios.

Hay muchos outliers a trabajar. 


In [ ]:
df_new = df_new.drop(['calculated_host_listings_count', 'reviews_per_month'], axis=1)

# Identificar las variables numéricas
variables_numericas = df_new._get_numeric_data().columns
print("Las variables numéricas son:")
for var in variables_numericas:
    print(f" - {var}")

# Identificar las variables categóricas
variables_categoricas = set(df_new.columns) - set(variables_numericas)
print("\nLas variables categóricas son:")
for var in variables_categoricas:
    print(f" - {var}")


### Análisis numérico-numérico


In [ ]:
pairs = list(combinations(variables_numericas, 2))  # Generar todas las combinaciones de pares

# Determinar el tamaño dinámico de la cuadrícula
rows = len(pairs) // 2 + len(pairs) % 2
fig, axes = plt.subplots(rows, 2, figsize=(15, rows * 3))

for idx, (col1, col2) in enumerate(pairs):
    row, col = divmod(idx, 2)  # Calcular posición de la cuadrícula
    sns.regplot(ax=axes[row, col], data=df_new, x=col1, y=col2, scatter_kws={'alpha': 0.5})
    axes[row, col].set_title(f'{col1} vs {col2}')
    corr = df_new[[col1, col2]].corr().iloc[0, 1]
    axes[row, col].annotate(f"Corr: {corr:.2f}", xy=(0.05, 0.95), xycoords='axes fraction', 
                            fontsize=10, color='red', ha='left', va='top')

# Eliminar espacios vacíos si no se usan todas las subplots
if len(pairs) % 2 != 0:
    fig.delaxes(axes[-1, -1])  # Eliminar el último eje si no hay suficientes pares

plt.tight_layout()
plt.show()


Conclusiones:

- Precio y Noches Mínimas: Estos datos no tienen una relación clara, por lo que probablemente no sea necesario realizar más análisis. Nada significativa.
- Precio y nº de reviews: Tiene una corelación muy floja, se ven acumuladas en precios más bajo. No significativo.
- Precio y disponibilidad: Parece haber una relación muy débil donde las propiedades más disponibles tienen precios más altos. No significativo.
- Mínimo de noches y reviews: Relación débil. Propiedades con menos noches mínimas pueden tener más reseñas. No significativo.
- Mínimo de noches y disponibilidad: Parece que cuanto más aumentan las noches también la disponibilidad, aunque hay outliers. Relación debil.
- Número de Reseñas y Disponibilidad: Existe una correlación leve que sugiere que la disponibilidad influye en el número de reseñas, pero no de manera muy significativa.

Podemos ver que entre las variables numéricas las correlaciones son muy bajas. Tienen a tener poca relación entre si. Y podemos observar los outliers que trabajar.

## 9. Análisis multivariante
#### Tras analizar las características una a una, es momento de analizarlas en relación con la predictoria y con ellas mismas para sacar conclusiones más claras acerca de sus relaciones y tomar decisiones sobre su procesamiento

In [ ]:
# Diccionario de colores personalizados
custom_palette = {
    'Bronx': '#1f77b4',
    'Brooklyn': '#ff7f0e',
    'Manhattan': '#2ca02c',
    'Queens': '#d62728',
    'Staten Island': '#9467bd'
}

sns.catplot(
    x='neighbourhood_group',
    y='price',
    data=df_new,
    kind='bar',
    hue='neighbourhood_group',
    palette=custom_palette
)

plt.title("Precio Promedio por Neighbourhood Group")
plt.xlabel("Neighbourhood Group")
plt.ylabel("Precio Promedio")
plt.show()


Hacemos este gráfico antes de factorizar para tener en cuenta que si que hay diferencias significativas y están relacionados los precios con los barrios

##### Factorizar las variables categóricas

In [ ]:
# Reemplazar las variables categóricas con sus valores numéricos /
for col in variables_categoricas:
    df_new[col] = pd.factorize(df_new[col])[0]

# Verificar el resultado
df_new.head()

#NO ESTA PERO ES MEJOR HACER COPIA DE LAS VARIABLES FACTORIZADAS Y MANTENER LA ORIGINAL

# Crear copias factorizadas de las columnas categóricas mientras se preservan las originales
# for col in variables_categoricas:
#   df_new[f"{col}_factorized"] = pd.factorize(df_new[col])[0]





Categorías de 'neighbourhood_group' y sus valores numéricos:  
  0 -> Brooklyn  
  1 -> Manhattan  
  2 -> Queens  
  3 -> Staten Island  
  4 -> Bronx  

Categorías de 'room_type' y sus valores numéricos:  
  0 -> Private room  
  1 -> Entire home/apt  
  2 -> Shared room  


In [ ]:
# Identificar las variables numéricas
variables_numericas = df_new._get_numeric_data().columns
print("Las variables numéricas son:")
for var in variables_numericas:
    print(f" - {var}")

# Identificar las variables categóricas
variables_categoricas = set(df_new.columns) - set(variables_numericas)
print("\nLas variables categóricas son:")
for var in variables_categoricas:
    print(f" - {var}")


In [ ]:
# Calcular la matriz de correlación entre todas las variables numéricas
correlation_matrix = df_new.corr()

# Graficar el heatmap para visualizar las correlaciones
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True)
plt.title("Matriz de Correlación entre Variables")
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

sns.boxplot(data=df_new, x='neighbourhood_group', y='price', ax=axes[0])
axes[0].set_title('Precio por Neighbourhood Group')

sns.boxplot(data=df_new, x='room_type', y='price', ax=axes[1])
axes[1].set_title('Precio por Room Type')

sns.boxplot(data=df_new, x='room_type', y='minimum_nights', ax=axes[2])
axes[2].set_title('Minimum Nights por Room Type')

plt.tight_layout()
plt.show()


In [ ]:
sns.catplot(x = 'neighbourhood_group', y = 'price', data = df_new)

In [ ]:
df1 =df_new[df_new['price']<500]
plt.figure(figsize = (10,5))
sns.violinplot(x = 'neighbourhood_group', y = 'price', data = df1, scale = 'count', linewidth = 0.3)

### Relaciones todos con todos

In [ ]:
# relaciones todos con todos
sns.pairplot(data = df_new)

## 10. Detectando y trabajando Outliers

In [ ]:
df_new.describe()

In [ ]:
# Detectar outliers usando el rango intercuartílico (IQR) y agregar información detallada
outliers_info = {}
for col in variables_numericas:
    Q1 = df_new[col].quantile(0.25)
    Q3 = df_new[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detectar los valores outliers
    outliers = df_new[(df_new[col] < lower_bound) | (df_new[col] > upper_bound)]
    num_outliers = len(outliers)
    
    # Guardar la información en el diccionario
    outliers_info[col] = {
        "num_outliers": num_outliers,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "outlier_range_below": outliers[col][outliers[col] < lower_bound].tolist(),
        "outlier_range_above": outliers[col][outliers[col] > upper_bound].tolist()
    }
    
    # Mostrar la información resumida para cada variable <3, qué bonito ha quedado
    print(f"Variable: {col}")
    print(f"  Outliers detectados: {num_outliers}")
    print(f"  Límite inferior: {lower_bound:.2f}")
    print(f"  Límite superior: {upper_bound:.2f}")
    print()  # Espacio para separar variables

#outliers_info  # Diccionario con toda la información de los outliers


In [ ]:
# Lista de colores para los boxplots
colors = sns.color_palette("pastel", len(variables_numericas))

# Configurar el layout de gráficos
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))  # Ajustar según el número de variables

# Iterar sobre las variables numéricas y graficar boxplots
for i, col in enumerate(variables_numericas):
    row, col_index = divmod(i, 3)  # Calcular posición en la cuadrícula
    sns.boxplot(data=df_new, y=col, ax=axes[row, col_index], color=colors[i])
    axes[row, col_index].set_title(f'Boxplot de {col}')
    axes[row, col_index].set_ylabel(col)

# Ajustar el diseño para evitar solapamientos
plt.tight_layout()
plt.show()


In [ ]:
df_new.head()

Creamos una copia del data set y uno lo trabajaremos con los outliers moldeados y otro con los datos originales.
Guardaremos los outliers originales en un archivo .json para recurrir a ellos en caso necesario. 

In [ ]:
# Ruta para guardar el archivo JSON
json_file_path = './data/outlier_limits.json'

# Crear un diccionario para almacenar los límites de outliers
outlier_limits = {}


# Guardar los límites en un archivo JSON
try:
    with open(json_file_path, 'w') as f:
        json.dump(outlier_limits, f, indent=4)
    print(f"Límites de outliers guardados en: {json_file_path}")
except Exception as e:
    print(f"No se pudo guardar el archivo JSON: {e}")





In [ ]:
# Crear DataFrame sin outliers
df_sin_outliers = df_new.copy()

In [ ]:
# Filtrar para eliminar los outliers según los límites calculados
for col in outlier_limits.keys():
    lower_bound = outlier_limits[col]["lower_bound"]
    upper_bound = outlier_limits[col]["upper_bound"]
    df_without_outliers = df_without_outliers[
        (df_sin_outliers[col] >= lower_bound) & (df_sin_outliers[col] <= upper_bound)
    ]

df_original = df_new # cambio el nomntr del df para luego aclararme

# Comparar tamaños
print("Tamaño original:", df_original.shape)
print("Tamaño sin outliers:", df_sin_outliers.shape)

## División de datos 

Cuando entrenamos modelos de Machine Learning necesitamos:

- Un conjunto de entrenamiento (train) para que el modelo aprenda los patrones de los datos
- Un conjunto de prueba (test) para evaluar el rendimiento del modelo en datos nuevos
  
Por lo general dividimos los datos en 80% para entrenamiento y un 20% para prueba.

#### 1. Identificar las variables predictorias y variable objetivo   
  
- Variable predictoria (x): Columnas que usará el modelo para predecir
- Variable objetivo (y): La que queremos predecir (en este caso 'price')
  
#### 2. Usar train_test_split (división automática en conjuntos de prueba y datos)


In [ ]:
x_sin_outliers = df_sin_outliers.drop(columns=['price']) #[aquí pondríamos las variables a trabajar en este caso no porque son todas numericas y trabajamos todas menos la objetiva]  #variable predictoria donde moldearemos outliers
x_original = df_original.drop(columns=['price']) # variable predictoria original
y = df_original['price'] #variable objetiva / esta no se toca 


# División 1: Conjunto original con outliers
x_train_con_outliers, x_test_con_outliers, y_train, y_test= train_test_split(x_original, y, test_size=0.2, random_state=42)

# División 2: Conjunto sin outliers
x_train_sin_outliers, x_test_sin_outliers, y_train, y_test= train_test_split(x_sin_outliers, y, test_size=0.2, random_state=42)

x_train_con_outliers.head()

In [ ]:
y_train.head()

## Implimentar Escalado y normalización

#### ¿Por qué hacer escalado y normalización?
  
- 'price' podría estar en miles, mientras que number_of_reviews puede estar entre 0 y 600.
- Esto puede llevar a que los modelos den más peso a las variables con valores más altos, afectando su rendimiento.
  
Para evitar esto, hacemos escalado o normalización. Esto asegura que todas las variables tengan un rango uniforme, ayudando al modelo a converger mejor y aprender más rápido.

![alt text](image.png)

##### NORMALIZACIÓN (Z-SCORE)

In [ ]:
#  Creamos escalador Z-Score 
scaler_norm = StandardScaler()

# Ajustar el escalador a los datos de entrenamiento
scaler_norm.fit(x_train_sin_outliers)

#Transformar los datos de entrenamiento 
x_train_norm = scaler_norm.transform(x_train_sin_outliers)
x_test_norm = scaler_norm.transform(x_test_sin_outliers)

# Convertir resultado en DFs
x_train_norm = pd.DataFrame(x_train_norm, index=x_train_sin_outliers.index, columns=x_train_sin_outliers.columns)
x_test_norm = pd.DataFrame(x_test_norm, index=x_test_sin_outliers.index, columns=x_test_sin_outliers.columns)


# Guardar el escalador
dump(scaler_norm, open("scaler_norm.sav", "wb"))


# Mostrar ejemplos de los datos normalizados
print("Datos normalizados - Entrenamiento:")
print(x_train_norm.head())




##### ESCALADO (Min-Max Scaling)



In [ ]:
# Crear el escalador Min-Max
scaler_minmax = MinMaxScaler()

# Ajustar el escalador a los datos de entrenamiento sin outliers
scaler_minmax.fit(x_train_sin_outliers)

# Ajustar a datos de entrenamiento
x_train_scaled = scaler_minmax.transform(x_train_sin_outliers)
x_test_scaled = scaler_minmax.transform(x_test_sin_outliers)

# Convertir los resultados a DF
x_train_scaled = pd.DataFrame(x_train_scaled, index=x_train_sin_outliers.index, columns=x_train_sin_outliers.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, index=x_test_sin_outliers.index, columns=x_test_sin_outliers.columns)
#guardar el escalador Min-Max
dump(scaler_minmax, open("scaler_minmax.sav", "wb"))

# Muestra
print("\nDatos escalados Min-Max - Entrenamiento:")
print(x_train_scaled.head())

## ENTRENAMIENTO DEL MODELO

# REGRESIÓN LINEAL

In [ ]:
# Crear un diccionario para almacenar resultados
resultados = {}

# 1. Modelo en datos originales (con outliers)
lr_con = LinearRegression()
lr_con.fit(x_train_con_outliers, y_train)
y_pred_con = lr_con.predict(x_test_con_outliers)
resultados["Con Outliers"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_con)),
    "R²": r2_score(y_test, y_pred_con),
}

# 2. Modelo en datos normalizados (Z-Score)
lr_norm = LinearRegression()
lr_norm.fit(x_train_norm, y_train)
y_pred_norm = lr_norm.predict(x_test_norm)
resultados["Normalizado (Z-Score)"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_norm)),
    "R²": r2_score(y_test, y_pred_norm),
}

# 3. Modelo en datos escalados (Min-Max)
lr_scaled = LinearRegression()
lr_scaled.fit(x_train_scaled, y_train)
y_pred_scaled = lr_scaled.predict(x_test_scaled)
resultados["Escalado (Min-Max)"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred_scaled)),
    "R²": r2_score(y_test, y_pred_scaled),
}

# Mostrar resultados
for conjunto, metricas in resultados.items():
    print(f"\nResultados para {conjunto}:")
    print(f" - RMSE: {metricas['RMSE']:.2f}")
    print(f" - R²: {metricas['R²']:.2f}")
    print("\nJAJAJAJA QUE PENA DE RESULTADOS PARA TANTO")

In [ ]:


# Entrenar un modelo Random Forest
rf = RandomForestRegressor(random_state=42)
rf.fit(x_train_con_outliers, y_train)
y_pred_rf = rf.predict(x_test_con_outliers)

# Evaluar el modelo
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("\nResultados para Random Forest:")
print(f" - RMSE: {rmse_rf:.2f}")
print(f" - R²: {r2_rf:.2f}")
